# CSI4142 - Data Science

Assignment 3 - Predictive analysis - Regression and Classification

Callum Frodsham 300199446

Name number

### Dataset Information
Dataset name: 

<a href="https://www.kaggle.com/datasets/mirichoi0218/insurance">Medical Cost Personal Datasets</a>
<br>
Provider: Miri Choi on Kaggle

In [ ]:
# Imports
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
# CHANGE THIS TO GITHUB RAW
data = pd.read_csv('insurance.csv')

In [ ]:
# Set display options to show full output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Data Cleaning
The dataset is already clean. This can be seen with the results of the code cell below: no entries are null, and all values are appropriate. No cleaning is needed.

In [ ]:
# Display information about the dataframe
print(data.info())

# Describe the dataframe
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Check for unique values in categorical columns to identify any unclean data
categorical_columns = data.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\nUnique values in column '{column}':\n", data[column].unique())

## 2. Categorical Feature Encoding
Through the get_dummies function, we can create the dataset with all numerical features becoming one-hot vectors. The rest of the features remain as they were.

In [ ]:
#print(data.head(), "\n")
# One-hot encode the specified features
data_onehot = pd.get_dummies(data, columns=['bmi', 'charges'])
print(data_onehot.head())

## EDA and Outlier detection
a. Outlier detection

In [ ]:
# Visualize categorical features
for column in categorical_columns:
    plt.figure(figsize=(10, 5))
    sns.countplot(data=data, x=column)
    plt.title(f'Distribution of {column}')
    plt.show()

It doesn't seem like outliers occur on categorical data.

In [ ]:
# Visualize numerical features
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
for column in numerical_columns:
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=data, x=column)
    plt.title(f'Box plot of {column}')
    plt.show()

In [ ]:
# Display the shape of the new dataframe
print("Original data shape:", data.shape)

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['bmi'].quantile(0.25)
Q3 = data['bmi'].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
data = data[(data['bmi'] >= lower_bound) & (data['bmi'] <= upper_bound)]

plt.figure(figsize=(10, 5))
sns.boxplot(data=data, x=data['bmi'])
plt.title(f'Box plot of BMI after outlier removal')
plt.show()

print("Data shape after removing outliers:", data.shape)